# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

349


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("connected to cluster")
except Exception as e:
    print("failed to connect to cluster")
    print(e)

connected to cluster


#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project_cassandra
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
    print("created workspace")

except Exception as e:
    print("failed to create workspace")
    print(e)

created workspace


#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("project_cassandra")
    print("project cassandra workspace success")
except Exception as e:
    print("project cassandra workspace failure")
    print(e)



project cassandra workspace success


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [8]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Slipknot,Aiden,M,0,Ramirez,192.57424,paid,"New York-Newark-Jersey City, NY-NJ-PA",19,Opium Of The People (Album Version),20
1,Cat Stevens,Aiden,M,1,Ramirez,170.57914,paid,"New York-Newark-Jersey City, NY-NJ-PA",19,Don't Be Shy,20
2,Collective Soul,Aiden,M,2,Ramirez,273.47546,paid,"New York-Newark-Jersey City, NY-NJ-PA",19,Run (LP Version),20
3,Taylor Swift,Aiden,M,3,Ramirez,233.89995,paid,"New York-Newark-Jersey City, NY-NJ-PA",19,Love Story,20
4,The Dixie Cups,Aiden,M,4,Ramirez,120.05832,paid,"New York-Newark-Jersey City, NY-NJ-PA",19,Iko Iko,20


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "CREATE TABLE IF NOT EXISTS session_item "
query1 = query1 + "( artist text, song text, length float, sessionId text, itemInSession text, \
                    PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1)
    print("session_item table created")
except Exception as e:
    print(f"failed to create session_item table: {e}")
                   

session_item table created


In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

# Reading csv file and inserting rows into cassandra tables.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item (artist, song, length, sessionId, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s) "
        session.execute(query, (line[0], line[10], float(line[5]), line[8],line[3]) )

This query "ALLOW Filtering" is run only to verify that sessionId=338 does not exist while itemInSession=4 exists

In [11]:
#validation purposes only
'''
select_query = "SELECT sessionId FROM session_item WHERE itemInSession='4' ALLOW FILTERING "

try:
    rows = session.execute(select_query)
except Exception as e:
    print(f"error getting query:{e}")
    
for row in rows:
    print(row)
'''

Row(sessionid='265')
Row(sessionid='117')
Row(sessionid='19')
Row(sessionid='350')
Row(sessionid='52')
Row(sessionid='376')
Row(sessionid='152')
Row(sessionid='439')
Row(sessionid='372')
Row(sessionid='199')
Row(sessionid='304')


### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4¶

As as a result  of sessionId=338 being absent, the first query returned no value

In [12]:
select_query = "SELECT  artist, song, length FROM session_item WHERE sessionId='338' AND itemInSession='4';"

try:
    rows = session.execute(select_query)
except Exception as e:
    print(f"error getting query:{e}")
    
for row in rows:
    print(row)


session Id was chosen as the primary key while the itemInSession was the composite key to optimize our data's partition across cassandra's nodes and potentially faster queries based on the questions of interest. As the data gets larger, say events data over 1 year, partitioning the data in this format may lead to faster queries.

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [13]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "CREATE TABLE IF  NOT EXISTS user_session"
query2 = query2 + "(userId text, sessionId text, itemInSession text, artist text, song text, \
                    firstName text, lastName text, PRIMARY KEY((userId, sessionId), itemInSession))"

try:
    session.execute(query2)
    print('Table user_session created')
except Exception as e:
    print(f"failed to create table user_session{e}")


                    

Table user_session created


In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song,firstName,lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s,%s,%s)"
        session.execute(query, (line[10], line[8], line[3], line[0], line[9], line[1],line[4]))


Note that "an allow filtering" query is only used to verify that sessionId=210 is not found in the session_item table while userId=10 exists

In [16]:
#validation purposes only
'''
validation_query = "SELECT artist, song, itemInSession, sessionId ,song, firstName, lastName FROM user_session \
                WHERE userId='10' ALLOW FILTERING"

try:
    rows = session.execute(validation_query)
except Exception as e:
    print(f"error getting query:{e}")
    
for row in rows:
    print(row)
'''


Row(artist='Klaus Badelt', song='Moonlight Serenade', iteminsession='1', sessionid='210', song_='Moonlight Serenade', firstname='Sylvie', lastname='Cruz')


In [15]:
select_query2 = "SELECT artist, song, itemInSession ,song, firstName, lastName FROM user_session \
                WHERE userId='10' AND sessionId='182'"

try:
    rows = session.execute(select_query2)
except Exception as e:
    print(f"error getting query:{e}")
    
for row in rows:
    print(row)

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "CREATE TABLE IF NOT EXISTS song"
query3 = query3 + "(song text, userId text, firstName text, lastName text,\
                    PRIMARY KEY(song, userId))"

try:
    session.execute(query3)
    print("created table song")
except Exception as e:
    print("Failed to create table song")
    print(e)
  

                    

created table song


In [17]:
file = "event_datafile_new.csv"

with open(file, encoding= "utf8") as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song (song, userId, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], line[10], line[1], line[4]))



In [18]:
select_query3 = "SELECT firstName, lastName from song WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(select_query3)
    print("query execute successfully")
except Exception as e:
    print(e)
    print("Failed to execute query")
    
for row in rows:
    print(row)


query execute successfully
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [19]:
## TO-DO: Drop the table before closing out the sessions

In [20]:
drop_query = "DROP TABLE IF EXISTS session_item; "

try:
    session.execute(drop_query)
    print("query execute successfully")
except Exception as e:
    print(e)
    print("Failed to execute query")


query execute successfully


In [21]:
drop_query = "DROP TABLE IF EXISTS user_session;"

try:
    session.execute(drop_query)
    print("query execute successfully")
except Exception as e:
    print(e)
    print("Failed to execute query")

query execute successfully


In [22]:
drop_query = "DROP TABLE IF EXISTS song;"

try:
    session.execute(drop_query)
    print("query execute successfully")
except Exception as e:
    print(e)
    print("Failed to execute query")

query execute successfully


### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()